In [1]:
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI
import warnings
from langchain.chains import LLMChain
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model = AzureChatOpenAI(temperature=0, deployment_name="chat")

# PromptTemplate

In [3]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {word1} joke about {word2}. Response in spanish"
)

In [4]:
chain = LLMChain(llm=model, prompt=prompt_template)
chain.run(word1="sports", word2="abejas")

'¿Qué hacen las abejas cuando juegan fútbol?\n\n¡Hacen colmena!'

In [5]:

from langchain_core.prompts import PromptTemplate
prompt_template = "Responde la siguiente pregunta: {question} de manera muy amable"
prompt = PromptTemplate(
    input_variables=["question"], template=prompt_template
)
llm = LLMChain(llm=model, prompt=prompt)

In [6]:
llm.run(question="Cómo puedo hacer pan amasado?")

'¡Claro! Aquí tienes una receta sencilla para hacer pan amasado:\n\nIngredientes:\n- 500 gramos de harina de trigo\n- 10 gramos de sal\n- 25 gramos de azúcar\n- 25 gramos de levadura fresca\n- 250 ml de agua tibia\n- 2 cucharadas de aceite de oliva\n\nPasos:\n1. En un recipiente grande, mezcla la harina, la sal y el azúcar.\n2. En otro recipiente pequeño, disuelve la levadura fresca en el agua tibia y deja reposar durante unos minutos hasta que se active.\n3. Agrega la levadura activada y el aceite de oliva a la mezcla de harina. Mezcla bien hasta formar una masa pegajosa.\n4. Espolvorea un poco de harina sobre una superficie limpia y amasa la masa durante unos 10 minutos, hasta que se vuelva suave y elástica.\n5. Coloca la masa en un recipiente ligeramente engrasado, cubre con un paño húmedo y deja reposar en un lugar cálido durante aproximadamente una hora, o hasta que la masa haya duplicado su tamaño.\n6. Precalienta el horno a 200 grados Celsius.\n7. Después de que la masa haya sub

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Debes responder la pregunta del usuario",
        ),
        ("human", "{question}"),
    ]
)
runnable = prompt | model | StrOutputParser()

for chunk in runnable.stream({"question": "Cómo puedo hacer pan amasado?"}):
    print(chunk, end="", flush=True)

Para hacer pan amasado, necesitarás los siguientes ingredientes:

- 500 gramos de harina de trigo
- 10 gramos de levadura fresca o 5 gramos de levadura seca
- 10 gramos de sal
- 300 ml de agua tibia
- 1 cucharada de aceite de oliva

Pasos para hacer el pan amasado:

1. En un recipiente grande, mezcla la harina y la sal. Haz un hueco en el centro de la harina y agrega la levadura desmenuzada o la levadura seca.

2. Agrega el agua tibia y el aceite de oliva al hueco que hiciste en la harina. Comienza a mezclar los ingredientes con una cuchara de madera o con tus manos.

3. Una vez que los ingredientes estén bien mezclados, comienza a amasar la masa sobre una superficie limpia y enharinada. Amasa durante unos 10-15 minutos, hasta que la masa esté suave y elástica.

4. Forma una bola con la masa y colócala en un recipiente ligeramente engrasado con aceite. Cubre el recipiente con un paño húmedo y deja que la masa repose en un lugar cálido durante aproximadamente una hora, o hasta que haya 

In [8]:
# FewPrompt
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {"input": "como", "output": "cómo"},
    {"input": "estas", "output": "estás"},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)



final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You should correct the user's spelling in Spanish."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)



chain = final_prompt | model | StrOutputParser()

chain.invoke({"input": "jugete"})


'juguete'

In [9]:
# Partial promptemplate
prompt = PromptTemplate(template="{strings_1}{strings_2}", input_variables=["strings_1", "strings_2"])
partial_prompt = prompt.partial(strings_1="foo")
print(partial_prompt)
print(partial_prompt.format(strings_2="baz"))

input_variables=['strings_2'] partial_variables={'strings_1': 'foo'} template='{strings_1}{strings_2}'
foobaz


In [10]:
# Inicializar
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
# print(prompt.format(bar="sa"))
# partial_prompt = prompt.partial(foo="ca")
partial_prompt = prompt.partial(bar="de")
partial_prompt

foosa


PromptTemplate(input_variables=[], partial_variables={'foo': 'foo', 'bar': 'de'}, template='{foo}{bar}')

In [11]:
from datetime import datetime
def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

prompt = PromptTemplate(
    template="El día de hoy {day} estamos a {date}",
    input_variables=["day", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(day="Martes"))


El día de hoy Martes estamos a 01/09/2024, 08:47:51


In [12]:
# Pipeline promt
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """Debes de ayudar a {person} a responder sus preguntas, pero con acento argentino."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """por ejemplo:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Ahora responde en argentino la siguiente pregunta:

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables

chain = pipeline_prompt | model | StrOutputParser()

chain.invoke({'example_a':"se dice vos", 'example_q':"como se dice tú en argentino", 'input':"como saludan en argentina", 'person':"Ignacia"})

'En Argentina, saludamos diciendo "¡Hola, che!"'